In [50]:
import math
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import numpy as np
np.random.seed(123)
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import LSTM
from keras.models import Sequential
from keras.utils import np_utils
from keras.models import Sequential
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [51]:
def load_dict_from_hdf5(filename):
    with h5py.File(filename, 'r') as h5file:
        return recursively_load_dict_contents_from_group(h5file, '/')
    h5file.close()

def recursively_load_dict_contents_from_group(h5file, path):
    ans = {}
    for key, item in h5file[path].items():
        if isinstance(item, h5py._hl.dataset.Dataset):
            ans[key] = item.value
        elif isinstance(item, h5py._hl.group.Group):
            ans[key] = recursively_load_dict_contents_from_group(h5file, path + key + '/')
    return ans
def convert_tensor(arg):
  return tf.convert_to_tensor(arg, dtype=tf.float32)

In [98]:
filename = "train.hdf5"
data = load_dict_from_hdf5(filename)

#get curves data
#get maximum size
maxi = -1
lenn = []
for a in data:
    for b in data[a]:
        #print(data[a][b]['label'])
        sample = list(list(data[a][b]['feat_bez_curves'].values())[0])
        sample = convert_tensor(sample)
        lenn.append(sample.shape[0])
        maxi = max(maxi, sample.shape[0])
        
#pad [602,0,0,0,0,0] into the tensors to convert them into (maxi, 9) shape
lst = []
for a in data:
    for b in data[a]:
        sample = list(list(data[a][b]['feat_bez_curves'].values())[0])
        sample = convert_tensor(sample)
        diff = maxi - sample.shape[0]
        t2 = []
        for c in range(diff):
            t2.append([0,0,0,0,0,0,0,0,2])
        if diff != 0:
            sample = tf.concat([sample, t2], 0)
        lst.append(sample)
#for dataA in data['0']:
#    sample = list(list(data['0'][dataA]['feat_bez_curves'].values())[0])
#    sample = convert_tensor(sample)
#    diff = maxi - sample.shape[0]
#    t2 = []
#    for a in range(diff):
#        t2.append([0,0,0,0,0,0,0,0,2])
#    if diff != 0:
#        sample = tf.concat([sample, t2], 0)
#    lst.append(sample)
    
#stack them together to get shape (N, maxi, 9)
curve = tf.stack(lst)
#curves = tf.RaggedTensor.from_tensor(curve, lengths = lenn)
curves = curve
curves = curves.numpy()
print(curves.shape)
#print(curves[1])
#print(curves[1].numpy())

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  # Remove the CWD from sys.path while we load stuff.


(295, 44, 9)


In [99]:
labels = []
for a in data:
    for b in data[a]:
        sample = data[a][b]['label']
        labels.append(sample)

values = array(labels)
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
Labels = onehot_encoded

//anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [104]:
#Labels = Labels.reshape(-1, 32, 12)
#curves = curves.reshape(-1, 32, 27, 9)
#tf.reshape(curves, (-1,32,27,9))
print(curves.shape)
print(Labels.shape)
#print(curves[0])
keras_model = tf.keras.Sequential([
    #tf.keras.layers.Input(shape=[None], dtype=tf.float32, ragged=True),
    tf.keras.layers.LSTM(42),
    tf.keras.layers.Dense(42, activation = "softmax"),
    #tf.keras.layers.Softmax(),
    #tf.keras.layers.Dense(75)
]
)
keras_model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
keras_model.fit(curves, Labels, epochs = 1000)
print(keras_model.summary())


(295, 44, 9)
(295, 42)
Epoch 1/1000
10/10 [==============================] - 0s 21ms/step - loss: 3.7155 - accuracy: 0.0068
Epoch 2/1000
10/10 [==============================] - 0s 22ms/step - loss: 3.6127 - accuracy: 0.0712
Epoch 3/1000
10/10 [==============================] - 0s 17ms/step - loss: 3.5169 - accuracy: 0.1085
Epoch 4/1000
10/10 [==============================] - 0s 16ms/step - loss: 3.4224 - accuracy: 0.1186
Epoch 5/1000
10/10 [==============================] - 0s 16ms/step - loss: 3.3428 - accuracy: 0.1458
Epoch 6/1000
10/10 [==============================] - 0s 16ms/step - loss: 3.2818 - accuracy: 0.1525
Epoch 7/1000
10/10 [==============================] - 0s 18ms/step - loss: 3.2210 - accuracy: 0.1559
Epoch 8/1000
10/10 [==============================] - 0s 22ms/step - loss: 3.1550 - accuracy: 0.1559
Epoch 9/1000
10/10 [==============================] - 0s 18ms/step - loss: 3.1038 - accuracy: 0.1695
Epoch 10/1000
10/10 [==============================] - 0s 17ms/step 

10/10 [==============================] - 0s 16ms/step - loss: 2.0113 - accuracy: 0.3051
Epoch 82/1000
10/10 [==============================] - 0s 16ms/step - loss: 2.0067 - accuracy: 0.3017
Epoch 83/1000
10/10 [==============================] - 0s 16ms/step - loss: 2.0189 - accuracy: 0.3017
Epoch 84/1000
10/10 [==============================] - 0s 16ms/step - loss: 2.0035 - accuracy: 0.3085
Epoch 85/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.9837 - accuracy: 0.3288
Epoch 86/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.9910 - accuracy: 0.3186
Epoch 87/1000
10/10 [==============================] - 0s 15ms/step - loss: 2.0213 - accuracy: 0.3017
Epoch 88/1000
10/10 [==============================] - 0s 16ms/step - loss: 2.0068 - accuracy: 0.3356
Epoch 89/1000
10/10 [==============================] - 0s 17ms/step - loss: 1.9922 - accuracy: 0.3458
Epoch 90/1000
10/10 [==============================] - 0s 20ms/step - loss: 1.9683 - accuracy: 0

10/10 [==============================] - 0s 16ms/step - loss: 1.7839 - accuracy: 0.4000
Epoch 161/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.7723 - accuracy: 0.3932
Epoch 162/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.7672 - accuracy: 0.4068
Epoch 163/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.7710 - accuracy: 0.3932
Epoch 164/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.7578 - accuracy: 0.4034
Epoch 165/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.7992 - accuracy: 0.3593
Epoch 166/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.7643 - accuracy: 0.3864
Epoch 167/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.7826 - accuracy: 0.3661
Epoch 168/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.7623 - accuracy: 0.3932
Epoch 169/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.7385 - ac

10/10 [==============================] - 0s 15ms/step - loss: 1.5909 - accuracy: 0.4712
Epoch 240/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.5830 - accuracy: 0.4746
Epoch 241/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.6215 - accuracy: 0.4508
Epoch 242/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.6237 - accuracy: 0.4441
Epoch 243/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.6195 - accuracy: 0.4339
Epoch 244/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.6213 - accuracy: 0.4271
Epoch 245/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.6131 - accuracy: 0.4475
Epoch 246/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.6079 - accuracy: 0.4576
Epoch 247/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.6048 - accuracy: 0.4441
Epoch 248/1000
10/10 [==============================] - 0s 19ms/step - loss: 1.6056 - ac

10/10 [==============================] - 0s 20ms/step - loss: 1.4536 - accuracy: 0.5051
Epoch 319/1000
10/10 [==============================] - 0s 21ms/step - loss: 1.4422 - accuracy: 0.5186
Epoch 320/1000
10/10 [==============================] - 0s 22ms/step - loss: 1.4397 - accuracy: 0.5017
Epoch 321/1000
10/10 [==============================] - 0s 18ms/step - loss: 1.4498 - accuracy: 0.5119
Epoch 322/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.4567 - accuracy: 0.5085
Epoch 323/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.4925 - accuracy: 0.5017
Epoch 324/1000
10/10 [==============================] - 0s 17ms/step - loss: 1.4649 - accuracy: 0.5051
Epoch 325/1000
10/10 [==============================] - 0s 22ms/step - loss: 1.4522 - accuracy: 0.4847
Epoch 326/1000
10/10 [==============================] - 0s 21ms/step - loss: 1.4409 - accuracy: 0.5254
Epoch 327/1000
10/10 [==============================] - 0s 17ms/step - loss: 1.4615 - ac

...

10/10 [==============================] - 0s 17ms/step - loss: 1.0758 - accuracy: 0.5898
Epoch 635/1000
10/10 [==============================] - 0s 19ms/step - loss: 1.0754 - accuracy: 0.5966
Epoch 636/1000
10/10 [==============================] - 0s 21ms/step - loss: 1.0878 - accuracy: 0.5966
Epoch 637/1000
10/10 [==============================] - 0s 17ms/step - loss: 1.0700 - accuracy: 0.6102
Epoch 638/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.0691 - accuracy: 0.6000
Epoch 639/1000
10/10 [==============================] - 0s 23ms/step - loss: 1.0621 - accuracy: 0.5898
Epoch 640/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.0547 - accuracy: 0.6136
Epoch 641/1000
10/10 [==============================] - 0s 21ms/step - loss: 1.0669 - accuracy: 0.6000
Epoch 642/1000
10/10 [==============================] - 0s 19ms/step - loss: 1.0627 - accuracy: 0.5864
Epoch 643/1000
10/10 [==============================] - 0s 18ms/step - loss: 1.0822 - ac

10/10 [==============================] - 0s 19ms/step - loss: 1.0463 - accuracy: 0.6102
Epoch 714/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.0589 - accuracy: 0.6034
Epoch 715/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.0519 - accuracy: 0.6034
Epoch 716/1000
10/10 [==============================] - 0s 17ms/step - loss: 1.0476 - accuracy: 0.6068
Epoch 717/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.0514 - accuracy: 0.6034
Epoch 718/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.0501 - accuracy: 0.6068
Epoch 719/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.0582 - accuracy: 0.6068
Epoch 720/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.0668 - accuracy: 0.5864
Epoch 721/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.1204 - accuracy: 0.5831
Epoch 722/1000
10/10 [==============================] - 0s 16ms/step - loss: 1.0680 - ac

10/10 [==============================] - 0s 15ms/step - loss: 1.0060 - accuracy: 0.6102
Epoch 793/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.9973 - accuracy: 0.6136
Epoch 794/1000
10/10 [==============================] - 0s 18ms/step - loss: 0.9856 - accuracy: 0.6136
Epoch 795/1000
10/10 [==============================] - 0s 17ms/step - loss: 0.9797 - accuracy: 0.6305
Epoch 796/1000
10/10 [==============================] - 0s 18ms/step - loss: 0.9790 - accuracy: 0.6305
Epoch 797/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.9922 - accuracy: 0.6169
Epoch 798/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.9986 - accuracy: 0.6271
Epoch 799/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.9962 - accuracy: 0.6136
Epoch 800/1000
10/10 [==============================] - 0s 15ms/step - loss: 1.0014 - accuracy: 0.6203
Epoch 801/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.9818 - ac

10/10 [==============================] - 0s 20ms/step - loss: 0.9356 - accuracy: 0.6441
Epoch 872/1000
10/10 [==============================] - 0s 19ms/step - loss: 0.9503 - accuracy: 0.6441
Epoch 873/1000
10/10 [==============================] - 0s 19ms/step - loss: 0.9305 - accuracy: 0.6271
Epoch 874/1000
10/10 [==============================] - 0s 19ms/step - loss: 0.9311 - accuracy: 0.6373
Epoch 875/1000
10/10 [==============================] - 0s 18ms/step - loss: 0.9203 - accuracy: 0.6203
Epoch 876/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.9329 - accuracy: 0.6373
Epoch 877/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.9441 - accuracy: 0.6373
Epoch 878/1000
10/10 [==============================] - 0s 20ms/step - loss: 0.9316 - accuracy: 0.6576
Epoch 879/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.9305 - accuracy: 0.6305
Epoch 880/1000
10/10 [==============================] - 0s 20ms/step - loss: 0.9181 - ac

10/10 [==============================] - 0s 17ms/step - loss: 0.9210 - accuracy: 0.6508 0s - loss: 0.9077 - accuracy: 0.
Epoch 951/1000
10/10 [==============================] - 0s 17ms/step - loss: 0.9075 - accuracy: 0.6407
Epoch 952/1000
10/10 [==============================] - 0s 17ms/step - loss: 0.8939 - accuracy: 0.6373
Epoch 953/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.8941 - accuracy: 0.6407
Epoch 954/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.8829 - accuracy: 0.6339
Epoch 955/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.8779 - accuracy: 0.6475
Epoch 956/1000
10/10 [==============================] - 0s 17ms/step - loss: 0.9071 - accuracy: 0.6508
Epoch 957/1000
10/10 [==============================] - 0s 23ms/step - loss: 0.9458 - accuracy: 0.6407
Epoch 958/1000
10/10 [==============================] - 0s 19ms/step - loss: 0.9122 - accuracy: 0.6475
Epoch 959/1000
10/10 [==============================] -